Cargo los datos sobre temperaturas durante el año 2022 de Barcelona desde la estación de Drassanes que tiene un criterio de temperatura a nivel ciudad.

In [15]:
import pandas as pd

# Cargo los archivos CSV
df1 = pd.read_csv('temp_012022_062022.csv')
df2 = pd.read_csv('temp_072022_122022.csv')

# Concateno los DataFrames por filas
df_temp_2022 = pd.concat([df1, df2], axis=0, ignore_index=True)

df_temp_2022.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin
0,2022-01-01,0201D,"BARCELONA, PORT OL?MPIC",BARCELONA,26,"11,9","0,0","7,7",06:50,"16,1",14:40,20.0,"2,5","5,8",12:30,90,94.0,00:00,64.0,10:00
1,2022-01-02,0201D,"BARCELONA, PORT OL?MPIC",BARCELONA,26,"10,4","0,0","6,5",07:50,"14,3",11:40,20.0,"2,5","5,6",12:30,90,93.0,Varias,68.0,10:00
2,2022-01-03,0201D,"BARCELONA, PORT OL?MPIC",BARCELONA,26,"11,7","0,0","8,2",07:10,"15,2",14:50,26.0,"2,5","7,5",15:10,88,93.0,Varias,70.0,10:10
3,2022-01-04,0201D,"BARCELONA, PORT OL?MPIC",BARCELONA,26,"15,4","5,1","11,1",05:30,"19,7",14:20,26.0,"3,9","8,3",02:30,84,93.0,Varias,57.0,10:50
4,2022-01-05,0201D,"BARCELONA, PORT OL?MPIC",BARCELONA,26,"10,3","0,8","6,9",23:59,"13,7",00:20,9.0,"0,8","14,7",01:40,75,93.0,00:00,57.0,13:10


El dataframe está compuesto por 365 filas y 20 columnas

In [16]:
df_temp_2022.shape 

(365, 20)

Hay columnas que no contemplo utilizar por lo que decido eliminarlas para aligerar el dataframe.

In [17]:
df_temp_2022= df_temp_2022.drop(columns=['indicativo', 'nombre', 'hrMax', 'horaHrMax', 'hrMin', 'horaHrMin', 'altitud', 'dir', 'racha', 'horaracha', 'horatmin', 'horatmax'])

df_temp_2022.head()

,fecha,provincia,tmed,prec,tmin,tmax,velmedia,hrMedia
0,2022-01-01,BARCELONA,"11,9","0,0","7,7","16,1","2,5",90
1,2022-01-02,BARCELONA,"10,4","0,0","6,5","14,3","2,5",90
2,2022-01-03,BARCELONA,"11,7","0,0","8,2","15,2","2,5",88
3,2022-01-04,BARCELONA,"15,4","5,1","11,1","19,7","3,9",84
4,2022-01-05,BARCELONA,"10,3","0,8","6,9","13,7","0,8",75


Reviso las características de cada columna, observo varios detalles sobre ellas que considero tratar:

* `fecha` es del tipo `object` y el formato debería ser `datetime`
* Hay varias columas que aparecen como tipo `object` al utiizar una coma como decimal y las transformaré como float con punto.


In [18]:
df_temp_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fecha      365 non-null    object
 1   provincia  365 non-null    object
 2   tmed       364 non-null    object
 3   prec       363 non-null    object
 4   tmin       364 non-null    object
 5   tmax       364 non-null    object
 6   velmedia   181 non-null    object
 7   hrMedia    365 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 22.9+ KB


Realizo la transformación de la columna `fecha` al tipo `datetime`

In [19]:
df_temp_2022['fecha']= pd.to_datetime(df_temp_2022['fecha'], format='%Y-%m-%d')
print(df_temp_2022.dtypes)

fecha        datetime64[ns]
provincia            object
tmed                 object
prec                 object
tmin                 object
tmax                 object
velmedia             object
hrMedia               int64
dtype: object


Transformo las columnas 'tmed', 'prec', 'tmin', 'tmax' al tipo float, modificando el símbolo de coma por punto.

In [20]:
columnas_a_float = ['tmed', 'prec', 'tmin', 'tmax', 'velmedia']

# Reemplazo las comas por puntos para que pueda realizarse el cambio al tipo float
for col in columnas_a_float:
    df_temp_2022[col] = df_temp_2022[col].str.replace(',', '.')  

df_temp_2022[columnas_a_float] = df_temp_2022[columnas_a_float].astype(float)

print(df_temp_2022.dtypes)

fecha        datetime64[ns]
provincia            object
tmed                float64
prec                float64
tmin                float64
tmax                float64
velmedia            float64
hrMedia               int64
dtype: object


In [21]:
df_temp_2022.isnull().sum()

fecha          0
provincia      0
tmed           1
prec           2
tmin           1
tmax           1
velmedia     184
hrMedia        0
dtype: int64

Analizo los valores nulos de la variable `tmed`; observo que aparecen varias columnas en ese registros con valores faltantes.

In [22]:
nulos_tmed = df_temp_2022['tmed'].isnull()
df_temp_2022[nulos_tmed]

,fecha,provincia,tmed,prec,tmin,tmax,velmedia,hrMedia
200,2022-07-20,BARCELONA,NaN,0.0,NaN,NaN,NaN,69


La idea de la imputación en este caso es mantener la continuidad de la serie. Imputo los valores faltantes utilizando una función, que en caso de que encuentre un valor nulo en una columna, buscará el valor correspondiente en la fila anterior (día anterior). Si el valor anterior existe, se utiliza para reemplazar el valor nulo. 

In [23]:
# Ordeno el DataFrame por fecha para asegurar la correcta imputación
df_temp_2022 = df_temp_2022.sort_values('fecha')

# Selecciono las columnas a imputar 
columnas_a_imputar = ['tmed', 'tmin', 'tmax', 'prec', 'velmedia', 'hrMedia']

# Se itera sobre las filas y las columnas a imputar
for index, row in df_temp_2022.iterrows():
    for columna in columnas_a_imputar:
        if pd.isna(row[columna]):
            # Buscar el valor del día anterior en la misma columna
            valor_anterior = df_temp_2022.loc[index - 1, columna]
            # Imputar si el valor anterior existe
            if not pd.isna(valor_anterior):
                df_temp_2022.loc[index, columna] = valor_anterior

Compruebo que los valores ha sido imputados totalmente.

In [24]:
df_temp_2022.isnull().sum()

fecha        0
provincia    0
tmed         0
prec         0
tmin         0
tmax         0
velmedia     0
hrMedia      0
dtype: int64

- Renombro las columna `provincia` por `ciudad`.

In [25]:
df_temp_2022= df_temp_2022.rename(columns={'provincia': 'ciudad', 'hrMedia': 'hrmedia'})
df_temp_2022= df_temp_2022[['fecha', 'ciudad', 'tmin', 'tmax', 'tmed', 'prec','hrmedia', 'velmedia']]
df_temp_2022.head()


,fecha,ciudad,tmin,tmax,tmed,prec,hrmedia,velmedia
0,2022-01-01,BARCELONA,7.7,16.1,11.9,0.0,90,2.5
1,2022-01-02,BARCELONA,6.5,14.3,10.4,0.0,90,2.5
2,2022-01-03,BARCELONA,8.2,15.2,11.7,0.0,88,2.5
3,2022-01-04,BARCELONA,11.1,19.7,15.4,5.1,84,3.9
4,2022-01-05,BARCELONA,6.9,13.7,10.3,0.8,75,0.8


Codifico la columna `ciudad` utilizando el valor `1` para poder relacionarlo en la base de datos con la tabla que corresponda a las ciudades

In [26]:
mapeo_ciudad= {"BARCELONA": "1"}

df_temp_2022['ciudad']= df_temp_2022['ciudad'].map(mapeo_ciudad)
df_temp_2022.head()

,fecha,ciudad,tmin,tmax,tmed,prec,hrmedia,velmedia
0,2022-01-01,1,7.7,16.1,11.9,0.0,90,2.5
1,2022-01-02,1,6.5,14.3,10.4,0.0,90,2.5
2,2022-01-03,1,8.2,15.2,11.7,0.0,88,2.5
3,2022-01-04,1,11.1,19.7,15.4,5.1,84,3.9
4,2022-01-05,1,6.9,13.7,10.3,0.8,75,0.8


Exporto el dataframe a un archivo en formato `csv`

In [27]:
df_temp_2022.to_csv('temp_bcn_2022.csv', index= False, encoding='utf-8')

Resumen del dataframe creado:

- Se eliminaron las columnas 'indicativo', 'nombre', 'hrMax', 'horaHrMax', 'hrMin', 'horaHrMin', 'altitud', 'dir', 'racha', 'horaracha', 'horatmin', 'horatmax'.
- Se realizaron transformaciones de tipo en las columnas 'tmed', 'prec', 'tmin', 'tmax', 'velmedia'.
- Se transformó la columna 'fecha' a tipo datetime
- Se trataron los valores nulos mediante la imputación del registro del día anterior al faltante, buscando mantener la continuidad de la serie. No hay valores faltantes.
- Se renombró la columna 'provincia' por 'ciudad' y se asignó el valor 1 para que sea utilizado en un modelo de tablas relaciones como identificador(id).
- Se exporto el dataframe a un archivo en formato csv.